<h2 align='center'>Download Data</h2>

In [ ]:
import csv
import urllib.request
import zipfile
import os
from urllib.request import urlretrieve
from zipfile import ZipFile
import pandas as pd


class BankingData:
    def __init__(self, url, data_name):
        self.url = url
        self.data_name = data_name

    def extract_to_csv(self):
        # check if "bank_data" folder exists, if not, create it
        if not os.path.exists("bank_data"):
            os.mkdir("bank_data")
        # Retrieve the zip file from the url link
        file = os.path.basename(self.url)
        urlretrieve(self.url, file)
        # Extract the zip file's contents
        with ZipFile(file, "r") as zf:
            zf.extractall("bank_data")
        # The file containing our data
        csv_file_name = f"{self.data_name}.csv"
        # Data clean up
        df = pd.read_csv(f"bank_data/{csv_file_name}", sep=";")
        # Save the cleaned up CSV file
        df.to_csv(df.to_csv(f"{self.data_name}_cleaned.csv", index=False))


class MarketData:
    def __init__(self, url, output_folder):
        self.url = url
        self.output_folder = output_folder

    def extract_asc_to_csv(self):
        """
        This function extracts the banking data provided from PKDD.
        It downloads the ZIP file from the "url".
        Then, it converts the .asc files to the .csv format.
        The function outputs a folder with a name from output_folder.
        This created folder will be in the current directory.

        Args:
            url (str): the URL containing the public data
            output_folder (str): the name of the folder where
            files will be stored
        """

        # Columns to rename for district table
        district_column_names = [
            "district_id",
            "district_name",
            "region",
            "no_of_inhabitants",
            "no_of_municipalities_lt_499",
            "no_of_municipalities_500_1999",
            "no_of_municipalities_2000_9999",
            "no_of_municipalities_gt_10000",
            "no_of_cities",
            "ratio_of_urban_inhabitants",
            "average_salary",
            "unemployment_rate_95",
            "unemployment_rate_96",
            "no_of_entrepreneurs_per_1000_inhabitants",
            "no_of_committed_crimes_95",
            "no_of_committed_crimes_96",
        ]

        # Download the ZIP file
        zip_file_path, _ = urllib.request.urlretrieve(self.url)
        # Extract the ZIP file
        with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
            zip_ref.extractall(self.output_folder)
        # Process ASC files and convert them to CSV
        for file_name in zip_ref.namelist():
            if file_name.endswith(".asc"):
                asc_path = os.path.join(self.output_folder, file_name)
                csv_file_name = file_name[:-4] + ".csv"
                csv_path = os.path.join(self.output_folder, csv_file_name)
                with open(asc_path, "r") as asc_file, open(
                    csv_path, "w", newline=""
                ) as csv_file:
                    asc_reader = csv.reader(asc_file, delimiter=";")
                    csv_writer = csv.writer(csv_file, delimiter=",")
                    if file_name == "district.asc":
                        next(asc_reader)
                        new_header = district_column_names
                        csv_writer.writerow(new_header)
                        csv_writer.writerows(asc_reader)
                    else:
                        for row in asc_reader:
                            csv_writer.writerow(row)
                print(f"Converted {asc_path} to CSV.")
        print("All ASC files converted to CSV.")

In [ ]:

_ = BankingData("https://tinyurl.com/jb-bank", "bank")
_.extract_to_csv()

_ = MarketData("https://tinyurl.com/jb-bank-m", "expanded_data")
_.extract_asc_to_csv()

<h2 align='center'>Install JupySQL</h2>

In [ ]:
!pip install jupysql==0.9.0 toml duckdb-engine --quiet

<h2 align='center'>Create pyproject.toml file</h2>

<h2 align='center'>Load Extension</h2>

In [19]:
import toml
file=open("pyproject.toml","w")
data_dict={
    "tool.jupysql.SqlMagic".strip('"').strip("'"): {
        "displaycon": False,
        "feedback": True,
        'autopolars': True
    }
}
toml.dump(data_dict,file)
file.close()

In [18]:
# Loading in SQL extension
%reload_ext sql

Found pyproject.toml from '/Users/macpro/Documents/GitHub/sql/colabs/video-material'

<h2 align='center'>Initialize DuckDB instance</h2>

In [20]:
# Initiating a DuckDB database named 'bank.duck.db' to run our SQL queries on
%sql duckdb:///bank.duck.db

Switching to connection duckdb:///bank.duck.db

<h2 align='center'>Create tables</h2>


In [4]:
%%sql
DROP TABLE IF EXISTS s1.district;
DROP TABLE IF EXISTS bank;
DROP SCHEMA IF EXISTS s1;

Running query in 'duckdb:///bank.duck.db'

Success


In [5]:
%%sql
CREATE OR REPLACE TABLE bank AS
FROM read_csv_auto('bank_cleaned.csv', header=True, sep=',')

Running query in 'duckdb:///bank.duck.db'

Count


In [6]:
%%sql
CREATE SCHEMA s1;
CREATE TABLE s1.account AS
FROM read_csv_auto('expanded_data/account.csv', header=True, sep=',');
CREATE TABLE s1.district AS
FROM read_csv_auto('expanded_data/district.csv', header=True, sep=',');
CREATE TABLE s1.loan AS
FROM read_csv_auto('expanded_data/loan.csv', header=True, sep=',');

Running query in 'duckdb:///bank.duck.db'

Count


<h2 align='center'>Perform queries (SQL format by default)</h2>

In [30]:
%config SqlMagic.autopolars=False 
%reload_ext sql

Found pyproject.toml from '/Users/macpro/Documents/GitHub/sql/colabs/video-material'

In [31]:
%%sql --save loan_amount_district_id --no-execute
SELECT l.loan_id, 
        l.amount, 
        a.district_id 
FROM s1.loan AS l 
INNER JOIN s1.account AS a 
    ON l.account_id = a.account_id

Running query in 'duckdb:///bank.duck.db'

Skipping execution...

In [32]:
# Loading in SQL extension

%sql duckdb:///bank.duck.db

Switching to connection duckdb:///bank.duck.db

In [33]:
result = %sql SELECT * FROM loan_amount_district_id

Generating CTE with stored snippets: 'loan_amount_district_id'

Running query in 'duckdb:///bank.duck.db'

In [34]:
type(result)

sql.run.resultset.ResultSet

In [35]:
result

loan_id,amount,district_id
6077,79608,44
7284,52788,15
7121,21924,55
5754,23052,54
6895,41904,68
6202,65184,54
6316,76908,12
5325,105804,12
6888,39576,1
6647,208320,1


<h2 align='center'>Reload extension</h2>

In [36]:
%config SqlMagic.autopolars=True 
%reload_ext sql

Found pyproject.toml from '/Users/macpro/Documents/GitHub/sql/colabs/video-material'

<h2 align='center'>Perform queries and transform them to polars</h2>

In [37]:
%%sql --save loan_amount_district_id --no-execute
SELECT l.loan_id, 
        l.amount, 
        a.district_id 
FROM s1.loan AS l 
INNER JOIN s1.account AS a 
    ON l.account_id = a.account_id

Running query in 'duckdb:///bank.duck.db'

Skipping execution...

In [38]:
result = %sql SELECT * FROM loan_amount_district_id
type(result)

Generating CTE with stored snippets: 'loan_amount_district_id'

Running query in 'duckdb:///bank.duck.db'

polars.dataframe.frame.DataFrame

In [39]:
result

loan_id,amount,district_id
i64,i64,i64
5830,405780,62
6303,72408,31
5895,93960,45
6732,174048,47
7308,129408,67
6665,99300,59
5938,402000,4
5041,369000,47
5082,262980,13


<h2 align='center'>Reload extension</h2>

In [41]:
%config SqlMagic.autopolars=False 
%config SqlMagic.autopandas=True 
%reload_ext sql

Found pyproject.toml from '/Users/macpro/Documents/GitHub/sql/colabs/video-material'

In [42]:
%%sql --save loan_amount_district_id --no-execute
SELECT l.loan_id, 
        l.amount, 
        a.district_id 
FROM s1.loan AS l 
INNER JOIN s1.account AS a 
    ON l.account_id = a.account_id

Running query in 'duckdb:///bank.duck.db'

Skipping execution...

In [43]:
result = %sql SELECT * FROM loan_amount_district_id
type(result)

Generating CTE with stored snippets: 'loan_amount_district_id'

Running query in 'duckdb:///bank.duck.db'

NoneType

In [44]:
result